# Comparing Two Climate Models

In this notebook, I will be comparing two climate reanalysis models:

* NCEP-DOE Reanalysis 2: Surface
* ERA5

I will be looking at the following variables:

* Surface Pressure
* Mean Sea Level Pressure
* Total Column Water

The idea is simple: these two models should have very similar properties. I will be trying to user RBIG in order to assess how similar these models are. I'll be looking at the following IT measures

* Entropy
* Total Correlation
* Mutual Information

If these climate models are that similar, then they should exhibit similar IT measures.

## Data - Climate Models

In [2]:
import os, sys
cwd = os.getcwd()
source_path = f"{cwd}/../../../"
sys.path.insert(0, f'{source_path}')

import numpy as np


# Data Loaders
from src.data.climate.amip import DataDownloader
from src.data.climate.amip import DataLoader

# ESDC tools
sys.path.insert(0, f'/home/emmanuel/code/py_esdc')
from esdc.standardize import normalize_temporal
from esdc.grid import regrid_data

import cdsapi
from zipfile import ZipFile
import pandas as pd
import xarray as xr
from tqdm import tqdm
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
data_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/"
results_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/results/"
fig_path = f"/home/emmanuel/projects/2020_rbig_rs/reports/figures/climate/"

### Download Datasets

In [15]:
downloader = DataDownloader()

downloader.download_all()

2019-11-06 16:03:59,814 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip5-monthly-single-levels
2019-11-06 16:04:00,498 INFO Request is completed
2019-11-06 16:04:00,499 INFO Downloading http://136.156.133.46/cache-compute-0015/cache/data4/70c39ac0-958f-4b70-8ac1-0991f58ba384-psl_Amon_IPSL-CM5B-LR_amip_r1i1p1_197901-200812.nc to /home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/ipsl_cm5b_lr/amip_ipsl_cm5b_lr.zip (12.7M)
2019-11-06 16:04:01,760 INFO Download rate 10.1M/s


/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/ipsl_cm5b_lr/amip_ipsl_cm5b_lr.zip
Already nc...
Changing name: 
 /home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/ipsl_cm5b_lr/amip_ipsl_cm5b_lr.nc


### Load Datasets

## loader = DataLoader()

dataset = 'ipsl_cm5b_lr'

xr_data = loader.load_amip_data(dataset)

In [6]:
xr_data

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 96, time: 360)
Coordinates:
  * time       (time) object 1979-01-16 12:00:00 ... 2008-12-16 12:00:00
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
    psl        (time, lat, lon) float32 dask.array<chunksize=(360, 96, 96), meta=np.ndarray>
Attributes:
    institution:            IPSL (Institut Pierre Simon Laplace, Paris, France)
    institute_id:           IPSL
    experiment_id:          amip
    source:                 IPSL-CM5B-LR (2011) : atmos : LMDZ5 (LMDZ5_NPv3.1...
    model_id:               IPSL-CM5B-LR
    forcing:

### Testing

In [7]:
datasets = [
    "inmcm4",
    "access1_0",
    "bcc_csm1_1",
    "bcc_csm1_1_m",
    "bnu_esm",
    "giss_e2_r",
    "cnrm_cm5",
    "ipsl_cm5a_lr",
    "ipsl_cm5a_mr",
    "ipsl_cm5b_lr",
    "mpi_esm_lr",
    "mpi_esm_mr",
    "noresm1_m",
]

for idataset in datasets:
    print(idataset)
    data = loader.load_amip_data(idataset)
    print(data.psl.shape)
    assert(type(data) is xr.Dataset)

inmcm4
(360, 120, 180)
access1_0
(360, 145, 192)
bcc_csm1_1
(360, 64, 128)
bcc_csm1_1_m
(360, 160, 320)
bnu_esm
(360, 64, 128)
giss_e2_r
(1572, 90, 144)
cnrm_cm5
(360, 128, 256)
ipsl_cm5a_lr
(372, 96, 96)
ipsl_cm5a_mr
(720, 143, 144)
ipsl_cm5b_lr
(360, 96, 96)
mpi_esm_lr
(360, 96, 192)
mpi_esm_mr
(360, 96, 192)
noresm1_m
(360, 96, 144)
